In [8]:
import pandas as pd
import re
import swifter
import pickle

In [48]:
# readinag txt file and creating dataframe
data = pd.read_csv("train_data.csv" )
data.head()

,Column1,tweet,offensive,hate,vlg,vio
0,1.0,@USER ردينا ع التطنز 😏👊🏻,OFF,NOT_HS,NOT_VLG,NOT_VIO
1,2.0,وصارت فطاير البقالات غذاء صحي 👎🏻 URL,NOT_OFF,NOT_HS,NOT_VLG,NOT_VIO
2,3.0,@USER روحي لبريده تلقين اشباه كثير بس ماحد زيك...,OFF,NOT_HS,NOT_VLG,NOT_VIO
3,4.0,@USER مش باين حاجه خالص 😣<LF>مش عارف بقى 😔,NOT_OFF,NOT_HS,NOT_VLG,NOT_VIO
4,5.0,#اليوم_الاثنين<LF><LF>👏 يقولك :%90 من المسلمي...,NOT_OFF,NOT_HS,NOT_VLG,NOT_VIO


In [49]:
data.drop(columns='Column1',inplace=True)

## deal with nulls

In [50]:
data.isna().sum()

tweet        329
offensive    330
hate         330
vlg          330
vio          331
dtype: int64

In [51]:
len(data)

8887

In [53]:
data.dropna(inplace=True)
len(data)

8556

## deal with categorical data

In [54]:
data.offensive.value_counts(),data.hate.value_counts(),data.vlg.value_counts(),data.vio.value_counts()

(NOT_OFF    5491
 OFF        3065
 Name: offensive, dtype: int64,
 NOT_HS    7634
 HS6        436
 HS1        251
 HS3        139
 HS5         69
 HS2         27
 Name: hate, dtype: int64,
 NOT_VLG    8424
 VLG         132
 Name: vlg, dtype: int64,
 NOT_VIO    8496
 VIO          60
 Name: vio, dtype: int64)

In [55]:
data["offensive"].replace({"OFF": 1, "NOT_OFF": 0}, inplace=True)
data["vlg"].replace({"VLG": 1, "NOT_VLG": 0}, inplace=True)
data["vio"].replace({"VIO": 1, "NOT_VIO": 0}, inplace=True)

In [56]:
data.head()

,tweet,offensive,hate,vlg,vio
0,@USER ردينا ع التطنز 😏👊🏻,1,NOT_HS,0,0
1,وصارت فطاير البقالات غذاء صحي 👎🏻 URL,0,NOT_HS,0,0
2,@USER روحي لبريده تلقين اشباه كثير بس ماحد زيك...,1,NOT_HS,0,0
3,@USER مش باين حاجه خالص 😣<LF>مش عارف بقى 😔,0,NOT_HS,0,0
4,#اليوم_الاثنين<LF><LF>👏 يقولك :%90 من المسلمي...,0,NOT_HS,0,0


In [57]:
# 1 hot encoding for HS column
ht = pd.get_dummies(data.hate, prefix='HS')
data=pd.concat([data, ht], axis=1)
data.drop(columns='hate',inplace=True)
data.head()

,tweet,offensive,vlg,vio,HS_HS1,HS_HS2,HS_HS3,HS_HS5,HS_HS6,HS_NOT_HS
0,@USER ردينا ع التطنز 😏👊🏻,1,0,0,0,0,0,0,0,1
1,وصارت فطاير البقالات غذاء صحي 👎🏻 URL,0,0,0,0,0,0,0,0,1
2,@USER روحي لبريده تلقين اشباه كثير بس ماحد زيك...,1,0,0,0,0,0,0,0,1
3,@USER مش باين حاجه خالص 😣<LF>مش عارف بقى 😔,0,0,0,0,0,0,0,0,1
4,#اليوم_الاثنين<LF><LF>👏 يقولك :%90 من المسلمي...,0,0,0,0,0,0,0,0,1


In [58]:
data.to_csv('train_data_2.csv',index=False)

## remove useless strings [punc, numbers, tashkeel, extra white spaces]


In [81]:
data=pd.read_csv("train_data_2.csv")

In [82]:

def remove_useless_chars(string):
    
    result= string.strip()
    
    # mention may be valuable
    result = re.sub('@USER',' مستخدم ',result)
    
    result = re.sub('<LF>','',result)

    result = re.sub('URL','',result)

    # Remove punctuations
    result = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', result)  
    

    
    # Remove numbers
    result = re.sub("\d+", " ", result)
    
#     # remove english character
#     result = re.sub('[A-Za-z]+',' ',result)
    
    # Remove Tashkeel
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    result = re.sub(noise, '', result)
    
    # remove extra whitespace
    result = re.sub('\s+', ' ', result) 
    
    
    return result

In [83]:
data.head()

,tweet,offensive,vlg,vio,HS_HS1,HS_HS2,HS_HS3,HS_HS5,HS_HS6,HS_NOT_HS
0,@USER ردينا ع التطنز 😏👊🏻,1,0,0,0,0,0,0,0,1
1,وصارت فطاير البقالات غذاء صحي 👎🏻 URL,0,0,0,0,0,0,0,0,1
2,@USER روحي لبريده تلقين اشباه كثير بس ماحد زيك...,1,0,0,0,0,0,0,0,1
3,@USER مش باين حاجه خالص 😣<LF>مش عارف بقى 😔,0,0,0,0,0,0,0,0,1
4,#اليوم_الاثنين<LF><LF>👏 يقولك :%90 من المسلمي...,0,0,0,0,0,0,0,0,1


In [84]:
data.tweet= data.tweet.apply(lambda x:remove_useless_chars(x))
data.head()

,tweet,offensive,vlg,vio,HS_HS1,HS_HS2,HS_HS3,HS_HS5,HS_HS6,HS_NOT_HS
0,مستخدم ردينا ع التطنز 😏👊🏻,1,0,0,0,0,0,0,0,1
1,وصارت فطاير البقالات غذاء صحي 👎🏻,0,0,0,0,0,0,0,0,1
2,مستخدم روحي لبريده تلقين اشباه كثير بس ماحد ز...,1,0,0,0,0,0,0,0,1
3,مستخدم مش باين حاجه خالص 😣مش عارف بقى 😔,0,0,0,0,0,0,0,0,1
4,اليوم الاثنين👏 يقولك من المسلمين عندهم خاله ا...,0,0,0,0,0,0,0,0,1


In [139]:
data.to_csv('train_data_3.csv',index=False)

## remove franco ,translate english ,remove extra english char

In [2]:
data = pd.read_csv('train_data_3.csv')

In [46]:
data.sample(10)

,tweet,offensive,vlg,vio,HS_HS1,HS_HS2,HS_HS3,HS_HS5,HS_HS6,HS_NOT_HS
287,شكل انا الوحيده الي ما احب المأكولات البحريه و...,0,0,0,0,0,0,0,0,1
4067,صباح الرشح 😷😷,0,0,0,0,0,0,0,0,1
2989,مستخدم والله الحمير معروفين منهم 👊,1,0,0,0,0,0,0,0,1
7212,مامى 😷,0,0,0,0,0,0,0,0,1
7195,مستخدم دي انا مش بناهم ف اسبوع يا مفتري 😂👊🏻👊🏻👊🏻,1,0,0,0,0,0,0,0,1
8475,RT مستخدم الأغنية عبارة عن mood swings بطريقة ...,0,0,0,0,0,0,0,0,1
3514,مستخدم يليل الزق يليل التبن يليل توي مستانس ا...,1,0,0,0,0,0,0,0,1
644,مستخدم تعالي واعمل الي انت عاوزو 😂😂😂😂👊,0,0,0,0,0,0,0,0,1
444,مستخدم انتبه وانت تزرف😣😂😂,0,0,0,0,0,0,0,0,1
7731,بايع الكليجا اللهم عليك بهم😷,0,0,0,0,0,0,0,0,1


In [13]:
import sys
sys.path.append("DSAraby-master/src")

In [18]:
from dsaraby import DSAraby



In [8]:
#replace franco to arabic 
def franco_to_arabic(text):
    ds = DSAraby()
    return ds.transliterate(text)

In [13]:
data.sample(10)

,tweet,offensive,vlg,vio,HS_HS1,HS_HS2,HS_HS3,HS_HS5,HS_HS6,HS_NOT_HS
1091,القنصليه الكويتيه في دبي👎🏻,0,0,0,0,0,0,0,0,1
8464,مستخدم ماطقع منك إلا هم والي في المقطع وش يعتب...,1,0,0,0,0,0,0,0,1
3917,RT مستخدم هنا المخرج حمار🐴 لم يغطي الشيبس بالر...,1,0,0,0,0,0,0,0,1
4988,مستخدم أول كنا نقول فيه من الشباب خرفان وما ند...,1,0,0,0,0,0,0,1,0
4974,RT مستخدم من الاخر اذا سيرا راح احلمو نجيب شي ...,0,0,0,0,0,0,0,0,1
5139,قلة ادب شوي عليها😤,1,0,0,0,0,0,0,0,1
1625,اسواء شي لما تسوين شغله و تستنين ردت فعل حلوه ...,0,0,0,0,0,0,0,0,1
182,مستخدم الجوهره يتسع ل الف 😣 الف كذبه و تواردت,0,0,0,0,0,0,0,0,1
1129,RT مستخدم ﺎﻟﺣرب ﺧدعهہ ﯙﺎﻟﻣرﺟلهہ ﻣﻳزان​ﻣو ﮔل ﻣن...,0,0,0,0,0,0,0,0,1
4641,مستخدم صوتك 👎,0,0,0,0,0,0,0,0,1


In [10]:
def remove_usless_en(text):
    result = re.sub('[A-Za-z]+',' ',text)
    return result

In [11]:

data.tweet = data.tweet.swifter.apply(lambda x:franco_to_arabic(x))

Pandas Apply:   0%|          | 0/8556 [00:00<?, ?it/s]

In [ ]:
data.tweet = data.tweet.swifter.apply(lambda x:remove_usless_en(x))

## remove redundant hhhhhhh 

In [4]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)


In [5]:
data.tweet = data.tweet.swifter.apply(lambda x: remove_repeating_char(x))

Pandas Apply:   0%|          | 0/8556 [00:00<?, ?it/s]

In [6]:
data.to_csv('train_data_4.csv',index=False)

In [16]:
data=pd.read_csv('train_data_4.csv')

In [44]:
data.sample(10)

,tweet,offensive,vlg,vio,HS_HS1,HS_HS2,HS_HS3,HS_HS5,HS_HS6,HS_NOT_HS
6594,RT مستخدم يع🙉,1,0,0,0,0,0,0,0,1
687,داليا مبارك امانه ع ايش خاقين عشانه أمريكي بس 🔪,0,0,0,0,0,0,0,0,1
7318,قولي الي تقولينه🤷🏻‍♀️ مب متغيره على حد 🤷🏻‍♀️ ع...,0,0,0,0,0,0,0,0,1
4514,خبر عاجل الرياض غبار شديد اليوم الأثنين 😷🤧,0,0,0,0,0,0,0,0,1
7131,مستخدم في كل وقت خطير ومضر وطعمه شين 😠💔,0,0,0,0,0,0,0,0,1
3597,RT مستخدم وضع الفحله الحين عندي بالخاص قاعده د...,1,1,0,0,0,0,0,0,1
7371,شنو معنى تكالب مو كأنها يايه من كلب 🐕😆RT مستخد...,1,0,0,0,0,0,0,0,1
7024,RT مستخدم شهنق ال فتكلم 😣,0,0,0,0,0,0,0,0,1
1987,يحاولون ويحاولون ويحاولون وهم وبدون ماتكلف ع ن...,1,0,0,0,0,0,0,0,1
5252,خلاص واضحة 😷 WEChamber EliminationChamber,0,0,0,0,0,0,0,0,1


# covert emoji

In [9]:
data=pd.read_csv('train_data_4.csv')

In [12]:
data.head()

,tweet,offensive,vlg,vio,HS_HS1,HS_HS2,HS_HS3,HS_HS5,HS_HS6,HS_NOT_HS
0,مستخدم ردينا ع التطنز 😏👊🏻,1,0,0,0,0,0,0,0,1
1,وصارت فطاير البقالات غذاء صحي 👎🏻,0,0,0,0,0,0,0,0,1
2,مستخدم روحي لبريده تلقين اشباه كثير بس ماحد زي...,1,0,0,0,0,0,0,0,1
3,مستخدم مش باين حاجه خالص 😣مش عارف بقى 😔,0,0,0,0,0,0,0,0,1
4,اليوم الاثنين👏 يقولك من المسلمين عندهم خاله اس...,0,0,0,0,0,0,0,0,1


In [14]:
imo_dict = pickle.load(open("ar_emojies_dect.pkl", 'rb'))


In [15]:
def remove_emoji(string):
    #to avoid missing spaces between emoji and surround words
    chars=[char for char in string]
    for char in range(len(chars)):
        for imo in imo_dict:
            if chars[char]==imo:
                chars[char]=' '+imo_dict.get(imo)+' '
    result=''.join(chars)            
    return result

In [16]:
data.tweet=data.tweet.swifter.apply(lambda x: remove_emoji(x))
data.head()

Pandas Apply:   0%|          | 0/8556 [00:00<?, ?it/s]

,tweet,offensive,vlg,vio,HS_HS1,HS_HS2,HS_HS3,HS_HS5,HS_HS6,HS_NOT_HS
0,مستخدم ردينا ع التطنز وجه مبتسم باستهزاء قبض...,1,0,0,0,0,0,0,0,1
1,وصارت فطاير البقالات غذاء صحي ابهام متجه لاسفل 🏻,0,0,0,0,0,0,0,0,1
2,مستخدم روحي لبريده تلقين اشباه كثير بس ماحد زي...,1,0,0,0,0,0,0,0,1
3,مستخدم مش باين حاجه خالص وجه منصدم مش عارف بق...,0,0,0,0,0,0,0,0,1
4,اليوم الاثنين تصفيق يقولك من المسلمين عندهم خ...,0,0,0,0,0,0,0,0,1


In [18]:
data.to_csv("train_data_5.csv")

# translate English to arabic

In [25]:
data.tweet[5252]

'خلاص واضحة  وجه بقناع طبي  WEChamber EliminationChamber'

In [28]:

def remove_en(text):
    result = re.sub('[A-Za-z]+','',text)
    return result


In [29]:
remove_en(data.tweet[5252])

'خلاص واضحة  وجه بقناع طبي   '

In [30]:
data.tweet = data.tweet.swifter.apply(lambda x: remove_en(x))
data.tweet[5252]

Pandas Apply:   0%|          | 0/8556 [00:00<?, ?it/s]

'خلاص واضحة  وجه بقناع طبي   '

In [32]:
 def remove_strip(text):
        result=text.strip()
        result = re.sub('\s+', ' ', text) 
        return result



In [33]:
data.tweet = data.tweet.swifter.apply(lambda x: remove_strip(x))
data.tweet[5252]

Pandas Apply:   0%|          | 0/8556 [00:00<?, ?it/s]

'خلاص واضحة وجه بقناع طبي '

In [34]:
data.to_csv("train_data_6.csv")